# Eligibility Signposting API - Request Journey Demo

**A practical walkthrough showing how a real request flows through our API**

---

---

# Part 1: Request Flow - Lambda to Response

## The Journey of a Request

When a request arrives at our API, it follows this path:

```
AWS Lambda → Flask App → Eligibility Blueprint → EligibilityService → Response
```


In [135]:

# Diagram
print("Visual Diagram:")
print("-" * 70)
print()
print("  AWS Lambda Event")
print("       ↓")
print("  lambda_handler(event, context)")
print("       ↓")
print("  Mangum (ASGI ↔ Lambda adapter)")
print("       ↓")
print("  WsgiToAsgi (WSGI ↔ ASGI adapter)")
print("       ↓")
print("  Flask App")
print("       ├─ eligibility_blueprint (prefix: /api/signposting/eligibility)")
print("       │   ├─ @before_request hooks")
print("       │   ├─ GET /_status → api_status()")
print("       │   └─ GET /<nhs_number> → check_eligibility()")
print("       │")
print("       └─ error_handler → handle_exception()")
print("       ↓")
print("  JSON Response")
print("       ↓")
print("  [Reverse flow through adapters]")
print("       ↓")
print("  Lambda Response")
print()
print()

print("Why this architecture?")
print("-" * 70)
print("✅ Mangum: Makes Flask work in AWS Lambda (serverless)")
print("✅ WsgiToAsgi: Bridges WSGI (Flask) and ASGI (modern async)")
print("✅ Flask: Provides routing, middleware, request/response handling")
print("✅ Blueprints: Organize routes into logical modules")
print("✅ Result: Traditional Flask app runs serverlessly in Lambda!")

Visual Diagram:
----------------------------------------------------------------------

  AWS Lambda Event
       ↓
  lambda_handler(event, context)
       ↓
  Mangum (ASGI ↔ Lambda adapter)
       ↓
  WsgiToAsgi (WSGI ↔ ASGI adapter)
       ↓
  Flask App
       ├─ eligibility_blueprint (prefix: /api/signposting/eligibility)
       │   ├─ @before_request hooks
       │   ├─ GET /_status → api_status()
       │   └─ GET /<nhs_number> → check_eligibility()
       │
       └─ error_handler → handle_exception()
       ↓
  JSON Response
       ↓
  [Reverse flow through adapters]
       ↓
  Lambda Response


Why this architecture?
----------------------------------------------------------------------
✅ Mangum: Makes Flask work in AWS Lambda (serverless)
✅ WsgiToAsgi: Bridges WSGI (Flask) and ASGI (modern async)
✅ Flask: Provides routing, middleware, request/response handling
✅ Blueprints: Organize routes into logical modules
✅ Result: Traditional Flask app runs serverlessly in Lambda!


## The View Layer: Eligibility Blueprint

The Flask blueprint handles HTTP requests and orchestrates the response:

In [136]:
print("🌐 View Layer (eligibility.py):")
print()
print("Routes:")
print("  GET /_status          → Health check endpoint")
print("  GET /<nhs_number>     → Main eligibility check")
print()
print("Request Flow:")
print("  1. before_request() → Initializes audit context")
print("  2. @validate_request_params() → Validates input")
print("  3. check_eligibility() → Main handler")
print("     ├─ Extract query params (category, conditions, includeActions)")
print("     ├─ Call EligibilityService")
print("     ├─ Write audit log")
print("     └─ Return JSON response")

🌐 View Layer (eligibility.py):

Routes:
  GET /_status          → Health check endpoint
  GET /<nhs_number>     → Main eligibility check

Request Flow:
  1. before_request() → Initializes audit context
  2. @validate_request_params() → Validates input
  3. check_eligibility() → Main handler
     ├─ Extract query params (category, conditions, includeActions)
     ├─ Call EligibilityService
     ├─ Write audit log
     └─ Return JSON response


## Demo - let's run through some requests and responses 

## Setup

First, let's set up our environment, initialize the Flask app, and import the necessary modules:

In [137]:
# Add the source directory to the Python path
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd() / 'src'))

# Initialize Flask app to provide application context for validators
from flask import Flask, g

app = Flask(__name__)
app_context = app.app_context()
app_context.push()

# Initialize audit log in Flask's g object (needed for eligibility calculator)
from eligibility_signposting_api.audit.audit_models import AuditEvent
g.audit_log = AuditEvent()

# Import the validation code
from eligibility_signposting_api.common.request_validator import (
    validate_query_params,
    validate_nhs_number
)

print("✓ Flask app initialized")
print("✓ Audit context initialized")
print("✓ Setup complete")

✓ Flask app initialized
✓ Audit context initialized
✓ Setup complete


---

# Part 1: Request Validation

Every request must pass validation before reaching our business logic.

## Example 1: A Valid Request ✓

In [138]:
print("Testing a VALID request:")
print("=" * 60)
print()

# Good request parameters
good_path_nhs = "9000000009"
good_header_nhs = "9000000009"
good_query_params = {
    "category": "VACCINATIONS",
    "conditions": "RSV",
    "includeActions": "Y"
}

print(f"NHS Number (path): {good_path_nhs}")
print(f"NHS Number (header): {good_header_nhs}")
print(f"Query Parameters: {good_query_params}")
print()

# Validate NHS number
nhs_valid = validate_nhs_number(good_path_nhs, good_header_nhs)
print(f"NHS Number validation: {'✓ PASS' if nhs_valid else '✗ FAIL'}")

# Validate query parameters
params_valid, error = validate_query_params(good_query_params)
print(f"Query params validation: {'✓ PASS' if params_valid else '✗ FAIL'}")

if nhs_valid and params_valid:
    print()
    print("✅ Request is valid - proceeding to eligibility check")
else:
    print(f"❌ Request failed: {error}")

Testing a VALID request:

NHS Number (path): 9000000009
NHS Number (header): 9000000009
Query Parameters: {'category': 'VACCINATIONS', 'conditions': 'RSV', 'includeActions': 'Y'}

NHS Number validation: ✓ PASS
Query params validation: ✓ PASS

✅ Request is valid - proceeding to eligibility check


## Example 2: Invalid Request - NHS Number Mismatch ✗

In [139]:
print("Testing INVALID request - NHS number mismatch:")
print("=" * 60)
print()

bad_path_nhs = "9000000009"
bad_header_nhs = "9000000010"  # Different!
query_params = {
    "category": "VACCINATIONS",
    "conditions": "RSV",
    "includeActions": "Y"
}

print(f"NHS Number (path): {bad_path_nhs}")
print(f"NHS Number (header): {bad_header_nhs}")
print()

nhs_valid = validate_nhs_number(bad_path_nhs, bad_header_nhs)
print(f"NHS Number validation: {'✓ PASS' if nhs_valid else '✗ FAIL'}")

if not nhs_valid:
    print()
    print("❌ HTTP 401 Unauthorized")
    print("Error: You are not authorised to request information for the supplied NHS Number")

NHS number mismatch


Testing INVALID request - NHS number mismatch:

NHS Number (path): 9000000009
NHS Number (header): 9000000010

NHS Number validation: ✗ FAIL

❌ HTTP 401 Unauthorized
Error: You are not authorised to request information for the supplied NHS Number


## Example 3: Invalid Request - Bad Query Parameters ✗

In [140]:
import json

print("Testing INVALID request - bad query parameters:")
print("=" * 60)
print()

bad_query_params = {
    "category": "INVALID_CATEGORY",  # Not VACCINATIONS, SCREENING, or ALL
    "conditions": "RSV",
    "includeActions": "Y"
}

print(f"Query Parameters: {bad_query_params}")
print()

params_valid, error_response = validate_query_params(bad_query_params)
print(f"Query params validation: {'✓ PASS' if params_valid else '✗ FAIL'}")

if not params_valid:
    print()
    print("❌ HTTP 400 Bad Request")
    print()
    # Parse the Flask Response object to show details
    # error_response is a Flask Response object (body, status_code, headers)
    response_body = error_response.get_data(as_text=True)
    error_body = json.loads(response_body)
    
    if "issue" in error_body and len(error_body["issue"]) > 0:
        issue = error_body["issue"][0]
        print(f"Error: {issue.get('diagnostics', 'Invalid parameters')}")
    print("Valid categories: VACCINATIONS, SCREENING, ALL")

Invalid category query param: 'INVALID_CATEGORY'


Testing INVALID request - bad query parameters:

Query Parameters: {'category': 'INVALID_CATEGORY', 'conditions': 'RSV', 'includeActions': 'Y'}

Query params validation: ✗ FAIL

❌ HTTP 400 Bad Request

Error: INVALID_CATEGORY is not a category that is supported by the API
Valid categories: VACCINATIONS, SCREENING, ALL


---

# Part 2: Setting Up Test Data

Now let's create the data for our first scenario:
- **Person**: 76 years old, no RSV vaccination, Leeds area, RSV cohort member

## Creating a Person Record

In [141]:
from eligibility_signposting_api.model.person import Person
from datetime import date
from dateutil.relativedelta import relativedelta

print("Creating Person Record")
print("=" * 60)
print()

# Calculate date of birth for 76 year old
today = date.today()
dob = today - relativedelta(years=76)

# Create person data (as it would come from DynamoDB)
person_dynamodb_items = [
    {
        "NHS_NUMBER": "9000000009",
        "ATTRIBUTE_TYPE": "PERSON",
        "DATE_OF_BIRTH": dob.strftime("%Y%m%d"),
        "POSTCODE": "LS1 4AP",
    },
    {
        "NHS_NUMBER": "9000000009",
        "ATTRIBUTE_TYPE": "COHORTS",
        "COHORT_MEMBERSHIPS": [
            {
                "COHORT_LABEL": "RSV_COHORT",
                "DATE_JOINED": "20240901"
            }
        ]
    },
    {
        "NHS_NUMBER": "9000000009",
        "ATTRIBUTE_TYPE": "RSV",  # Target condition as ATTRIBUTE_TYPE
        # No LAST_SUCCESSFUL_DATE means never vaccinated
        # If vaccinated, would have: "LAST_SUCCESSFUL_DATE": "20230415" (YYYYMMDD format)
    }
]

# Create Person object
person = Person(data=person_dynamodb_items)

print("Person created with attributes:")
for item in person_dynamodb_items:
    print(f"  {item['ATTRIBUTE_TYPE']}:")
    for key, value in item.items():
        if key not in ['NHS_NUMBER', 'ATTRIBUTE_TYPE']:
            print(f"    • {key}: {value}")
    print()

print("✓ Person record created")

Creating Person Record

Person created with attributes:
  PERSON:
    • DATE_OF_BIRTH: 19491012
    • POSTCODE: LS1 4AP

  COHORTS:
    • COHORT_MEMBERSHIPS: [{'COHORT_LABEL': 'RSV_COHORT', 'DATE_JOINED': '20240901'}]

  RSV:

✓ Person record created


## Creating an RSV Campaign Configuration

This campaign uses rules that reflect, but don't match, the actual production configuration:
- **Filter**: Age check using DATE_OF_BIRTH (must be 75+)
- **Filter**: Vaccination history check (exclude if vaccinated in last 25 years)
- **Suppression**: Shows message if already vaccinated
- **Redirect**: Leeds residents directed to contact GP

In [142]:
from eligibility_signposting_api.model.campaign_config import (
    CampaignConfig,
    Iteration,
    IterationCohort,
    IterationRule,
    AvailableAction,
    ActionsMapper,
    RuleType,
    RuleOperator,
    RuleAttributeLevel
)
from datetime import date, timedelta

print("Creating RSV Campaign Configuration")
print("=" * 60)
print()

# Define campaign dates and iteration
# Make sure campaign is "live" by including today's date
today = date.today()
campaign_start = today - timedelta(days=30)  # Started 30 days ago
campaign_end = today + timedelta(days=150)   # Ends in 150 days
iteration_date = campaign_start

print(f"Campaign period: {campaign_start} to {campaign_end}")
print(f"Today: {today} (campaign is LIVE)")
print()

# Create actions mapper
actions_mapper = ActionsMapper({
    "CONTACT_GP_LEEDS": AvailableAction(
        ActionType="CONTACT",
        ExternalRoutingCode="CONTACT_GP_RSV",
        ActionDescription="Contact your GP surgery to arrange RSV vaccination",
        UrlLink="https://www.nhs.uk/service-search/find-a-gp",
        UrlLabel="Find your GP"
    ),
    "INFO_RSV": AvailableAction(
        ActionType="INFO",
        ExternalRoutingCode="INFO_RSV",
        ActionDescription="Learn more about RSV vaccination",
        UrlLink="https://www.nhs.uk/conditions/vaccinations/rsv-vaccine",
        UrlLabel="Learn more"
    )
})

# Create campaign rules using IterationRule
# Note: Rules follow the actual config structure from production
rules = [
    # Filter rule: Age >= 75 (using DATE_OF_BIRTH with year operator)
    IterationRule(
        Name="Remove under 75 Years",
        Type=RuleType.filter,
        Priority=10,
        AttributeName="DATE_OF_BIRTH",
        Operator=RuleOperator.year_gt,  # Y> means "years greater than"
        Comparator="-75",  # -75 years from today
        AttributeTarget="",
        Description="Ensure anyone who has a DOB determining age < 75 is filtered out",
        AttributeLevel=RuleAttributeLevel.PERSON,
        CohortLabel="RSV_COHORT",
        RuleStop="N"
    ),
    # Filter rule: Not already vaccinated (checks LAST_SUCCESSFUL_DATE)
    IterationRule(
        Name="Remove from RSV cohort if already vaccinated",
        Type=RuleType.filter,
        Priority=20,
        AttributeName="LAST_SUCCESSFUL_DATE",
        Operator=RuleOperator.year_gte,  # Y>= means "years greater than or equal"
        Comparator="-25",  # Vaccinated within last 25 years
        AttributeTarget="RSV",
        Description="Remove anyone already vaccinated from RSV cohort",
        AttributeLevel=RuleAttributeLevel.TARGET,
        CohortLabel="RSV_COHORT",
        RuleStop="N"
    ),
    # Suppression rule: Already vaccinated (with user-facing message)
    IterationRule(
        Name="AlreadyVaccinated",
        Type=RuleType.suppression,
        Priority=30,
        AttributeName="LAST_SUCCESSFUL_DATE",
        Operator=RuleOperator.year_gte,
        Comparator="-25",
        AttributeTarget="RSV",
        Description="## You've had your RSV vaccination\n\nOur records show you were vaccinated against RSV.",
        AttributeLevel=RuleAttributeLevel.TARGET,
        RuleStop="Y"  # Stop processing if matched
    ),
    # Redirect rule: Leeds area - contact GP
    IterationRule(
        Name="LeedsAreaContactGP",
        Type=RuleType.redirect,
        Priority=100,
        AttributeName="POSTCODE",
        Operator=RuleOperator.starts_with,
        Comparator="LS",  # Comparator is the VALUE to compare against
        AttributeTarget="",
        Description="Leeds residents should contact their GP",
        AttributeLevel=RuleAttributeLevel.PERSON,
        CommsRouting="CONTACT_GP_LEEDS",
        RuleStop="N"
    )
]

# Create iteration with cohorts
iteration = Iteration(
    ID="RSV_2024_ITER_001",
    Version=1,
    Name="RSV_Winter_2024",
    IterationDate=iteration_date,
    Type="A",
    DefaultCommsRouting="INFO_RSV",
    DefaultNotEligibleRouting="INFO_RSV",
    DefaultNotActionableRouting="INFO_RSV",
    IterationCohorts=[
        IterationCohort(
            CohortLabel="RSV_COHORT",
            CohortGroup="Older Adults"
        )
    ],
    IterationRules=rules,
    ActionsMapper=actions_mapper
)

# Create campaign config
campaign_config = CampaignConfig(
    ID="RSV_WINTER_2024",
    Version=1,
    Name="RSV_WINTER_2024",
    Type="V",
    Target="RSV",
    IterationFrequency="A",
    IterationType="A",
    StartDate=campaign_start,
    EndDate=campaign_end,
    Iterations=[iteration]
)

print("Campaign Configuration Created:")
print(f"  Campaign: {campaign_config.name}")
print(f"  Condition: {campaign_config.target}")
print(f"  Period: {campaign_start} to {campaign_end}")
print(f"  Campaign Live: {campaign_config.campaign_live}")
print(f"  Rules: {len(rules)}")
print()
print("Rules:")
for rule in rules:
    print(f"  • {rule.name} ({rule.type.value}) - Priority {rule.priority}")
    print(f"    Check: {rule.attribute_name} {rule.operator.value} {rule.attribute_target}")
print()
print("Actions available:")
for code, action in actions_mapper.root.items():
    print(f"  • {code}: {action.action_description}")
print()
print("✓ Campaign configuration created")

Creating RSV Campaign Configuration

Campaign period: 2025-09-12 to 2026-03-11
Today: 2025-10-12 (campaign is LIVE)

Campaign Configuration Created:
  Campaign: RSV_WINTER_2024
  Condition: RSV
  Period: 2025-09-12 to 2026-03-11
  Campaign Live: True
  Rules: 4

Rules:
  • Remove under 75 Years (F) - Priority 10
    Check: DATE_OF_BIRTH Y> 
  • Remove from RSV cohort if already vaccinated (F) - Priority 20
    Check: LAST_SUCCESSFUL_DATE Y>= RSV
  • AlreadyVaccinated (S) - Priority 30
    Check: LAST_SUCCESSFUL_DATE Y>= RSV
  • LeedsAreaContactGP (R) - Priority 100
    Check: POSTCODE starts_with 

Actions available:
  • CONTACT_GP_LEEDS: Contact your GP surgery to arrange RSV vaccination
  • INFO_RSV: Learn more about RSV vaccination

✓ Campaign configuration created


---

# Part 3: Running the Eligibility Service

Now we'll run the actual eligibility calculation using our real service code.

## Create the Eligibility Calculator

In [144]:
from eligibility_signposting_api.services.calculators.eligibility_calculator import (
    EligibilityCalculatorFactory
)

print("Creating Eligibility Calculator")
print("=" * 60)
print()

# Create calculator with person and campaign data
calculator = EligibilityCalculatorFactory.get(
    person=person,
    campaign_configs=[campaign_config]
)

print("✓ Calculator created with:")
print(f"  • Person: NHS Number {person_dynamodb_items[0]['NHS_NUMBER']}")
print(f"  • Campaigns: {len([campaign_config])}")

Creating Eligibility Calculator

✓ Calculator created with:
  • Person: NHS Number 9000000009
  • Campaigns: 1


## Calculate Eligibility Status

In [145]:
from eligibility_signposting_api.model.eligibility_status import NHSNumber, ConditionName

print("Running Eligibility Calculation")
print("=" * 60)
print()

# Get eligibility status (same as what EligibilityService would do)
eligibility_status = calculator.get_eligibility_status(
    include_actions="Y",
    conditions=["RSV"],
    category="VACCINATIONS"
)

print("✓ Eligibility calculated")
print()
print("Processing what happened...")

Running Eligibility Calculation

✓ Eligibility calculated

Processing what happened...


## Examine the Results

In [146]:
print("Eligibility Calculation Results")
print("=" * 60)
print()

# Debug: Check what we got back
print(f"Number of conditions in result: {len(eligibility_status.conditions)}")
print(f"Eligibility status object: {eligibility_status}")
print()

# Check if we have any conditions
if not eligibility_status.conditions:
    print("❌ ERROR: No conditions returned from eligibility calculation")
    print("This might mean the campaign configuration doesn't match the requested condition.")
else:
    print(f"✓ Found {len(eligibility_status.conditions)} condition(s)")

print()
print("=" * 60)
print()

# Only proceed if we have conditions
if eligibility_status.conditions:
    # Get the RSV condition result
    rsv_condition = eligibility_status.conditions[0]

    print(f"Condition: {rsv_condition.condition_name}")
    print(f"Status: {rsv_condition.status.name.upper()}")
    print(f"Status Text: {rsv_condition.status_text}")
    print()

    # Show which rules were evaluated (suitability_rules)
    print("Rules Evaluated:")
    for reason in rsv_condition.suitability_rules:
        matched_symbol = "✓" if reason.matcher_matched else "✗"
        matched_text = "MATCHED" if reason.matcher_matched else "NOT MATCHED"
        print(f"  {matched_symbol} {reason.rule_name} ({reason.rule_type.value}): {matched_text}")
        if reason.rule_description:
            print(f"      {reason.rule_description}")
    print()

    # Show suggested actions
    if rsv_condition.actions:
        print(f"Suggested Actions ({len(rsv_condition.actions)}):")
        for action in rsv_condition.actions:
            print(f"  • {action.action_type}: {action.action_description}")
            if action.url_link:
                print(f"    Link: {action.url_link}")
                if action.url_label:
                    print(f"    Label: {action.url_label}")
    else:
        print("No actions suggested")

    print()
    print("=" * 60)
    print()

    # Summary
    if rsv_condition.status.name == "actionable":
        print("✅ RESULT: Person is ELIGIBLE for RSV vaccination")
        print(f"   They should: {rsv_condition.actions[0].action_description if rsv_condition.actions else 'See details above'}")
    elif rsv_condition.status.name == "not_actionable":
        print("⚠️  RESULT: Person is eligible but NOT ACTIONABLE")
    else:
        print("❌ RESULT: Person is NOT ELIGIBLE for RSV vaccination")

Eligibility Calculation Results

Number of conditions in result: 1
Eligibility status object: EligibilityStatus(conditions=[Condition(condition_name='RSV', status=<Status.actionable: 3>, cohort_results=[CohortGroupResult(cohort_code='Older Adults', status=<Status.actionable: 3>, reasons=[], description='', audit_rules=[])], suitability_rules=[], status_text='You should have the RSV vaccine', actions=[SuggestedAction(action_type='CONTACT', action_code='CONTACT_GP_RSV', action_description='Contact your GP surgery to arrange RSV vaccination', url_link=HttpUrl('https://www.nhs.uk/service-search/find-a-gp'), url_label='Find your GP', internal_action_code='CONTACT_GP_LEEDS')])])

✓ Found 1 condition(s)


Condition: RSV
Status: ACTIONABLE
Status Text: You should have the RSV vaccine

Rules Evaluated:

Suggested Actions (1):
  • CONTACT: Contact your GP surgery to arrange RSV vaccination
    Link: https://www.nhs.uk/service-search/find-a-gp
    Label: Find your GP


✅ RESULT: Person is ELIGIBL

## JSON Response Format

This is what would be returned to the API caller:

In [ ]:
import json

print("API Response (JSON):")
print("=" * 60)
print()

# Convert to dict (similar to what the view layer does)
# Note: In the actual API, this goes through additional response models
response_data = {
    "conditions": [
        {
            "condition": rsv_condition.condition_name,
            "status": rsv_condition.status.name,
            "statusText": rsv_condition.status_text,
            "reasons": [
                {
                    "ruleType": reason.rule_type.value,
                    "ruleName": reason.rule_name,
                    "matched": reason.matcher_matched,
                    "description": reason.rule_description
                }
                for reason in rsv_condition.suitability_rules
            ],
            "suggestedActions": [
                {
                    "actionType": action.action_type,
                    "actionCode": action.action_code,
                    "description": action.action_description,
                    "link": str(action.url_link) if action.url_link else None,
                    "linkLabel": action.url_label if action.url_label else None
                }
                for action in (rsv_condition.actions or [])
            ]
        }
    ]
}

print(json.dumps(response_data, indent=2))
print()
print("This would be returned as HTTP 200 with Content-Type: application/fhir+json")

API Response (JSON):

{
  "conditions": [
    {
      "condition": "RSV",
      "status": "actionable",
      "statusText": "You should have the RSV vaccine",
      "reasons": [],
      "suggestedActions": [
        {
          "actionType": "CONTACT",
          "actionCode": "CONTACT_GP_RSV",
          "description": "Contact your GP surgery to arrange RSV vaccination",
          "link": "https://www.nhs.uk/service-search/find-a-gp",
          "linkLabel": "Find your GP"
        }
      ]
    }
  ]
}

This would be returned as HTTP 200 with Content-Type: application/json


---

# Example 2: Not Eligible Person (Already Vaccinated)

Now let's test someone who **already had their RSV vaccination** to see how the suppression rule works.

## Create a Vaccinated Person

In [148]:
print("Creating Person Record - Already Vaccinated")
print("=" * 60)
print()

# Calculate date of birth for 78 year old
today = date.today()
dob_vaccinated = today - relativedelta(years=78)

# This person was vaccinated 6 months ago
vaccination_date = today - relativedelta(months=6)

# Create person data with vaccination history
person_vaccinated_items = [
    {
        "NHS_NUMBER": "9000000010",
        "ATTRIBUTE_TYPE": "PERSON",
        "DATE_OF_BIRTH": dob_vaccinated.strftime("%Y%m%d"),
        "POSTCODE": "M1 1AE",  # Manchester area
    },
    {
        "NHS_NUMBER": "9000000010",
        "ATTRIBUTE_TYPE": "COHORTS",
        "COHORT_MEMBERSHIPS": [
            {
                "COHORT_LABEL": "RSV_COHORT",
                "DATE_JOINED": "20240901"
            }
        ]
    },
    {
        "NHS_NUMBER": "9000000010",
        "ATTRIBUTE_TYPE": "RSV",
        "LAST_SUCCESSFUL_DATE": vaccination_date.strftime("%Y%m%d"),  # Vaccinated 6 months ago
    }
]

# Create Person object
person_vaccinated = Person(data=person_vaccinated_items)

print("Person created with attributes:")
for item in person_vaccinated_items:
    print(f"  {item['ATTRIBUTE_TYPE']}:")
    for key, value in item.items():
        if key not in ['NHS_NUMBER', 'ATTRIBUTE_TYPE']:
            print(f"    • {key}: {value}")
    print()

print(f"✓ Person record created (vaccinated on {vaccination_date.strftime('%Y-%m-%d')})")

Creating Person Record - Already Vaccinated

Person created with attributes:
  PERSON:
    • DATE_OF_BIRTH: 19471012
    • POSTCODE: M1 1AE

  COHORTS:
    • COHORT_MEMBERSHIPS: [{'COHORT_LABEL': 'RSV_COHORT', 'DATE_JOINED': '20240901'}]

  RSV:
    • LAST_SUCCESSFUL_DATE: 20250412

✓ Person record created (vaccinated on 2025-04-12)


## Calculate Eligibility for Vaccinated Person

In [149]:
print("Running Eligibility Calculation - Vaccinated Person")
print("=" * 60)
print()

# Create calculator with vaccinated person and same campaign config
calculator_vaccinated = EligibilityCalculatorFactory.get(
    person=person_vaccinated,
    campaign_configs=[campaign_config]
)

# Get eligibility status
eligibility_status_vaccinated = calculator_vaccinated.get_eligibility_status(
    include_actions="Y",
    conditions=["RSV"],
    category="VACCINATIONS"
)

print("✓ Eligibility calculated")
print()

Running Eligibility Calculation - Vaccinated Person

✓ Eligibility calculated



## Examine Results - Not Eligible

In [152]:
print("Eligibility Calculation Results - Vaccinated Person")
print("=" * 60)
print()

if eligibility_status_vaccinated.conditions:
    rsv_condition_vaccinated = eligibility_status_vaccinated.conditions[0]

    print(f"Condition: {rsv_condition_vaccinated.condition_name}")
    print(f"Status: {rsv_condition_vaccinated.status.name.upper()}")
    print(f"Status Text: {rsv_condition_vaccinated.status_text}")
    print()

    # Show which rules were evaluated
    print("Rules Evaluated:")
    for reason in rsv_condition_vaccinated.suitability_rules:
        matched_symbol = "✓" if reason.matcher_matched else "✗"
        matched_text = "MATCHED" if reason.matcher_matched else "NOT MATCHED"
        print(f"  {matched_symbol} {reason.rule_name} ({reason.rule_type.value}): {matched_text}")
        if reason.rule_description:
            # Show first line of description (may be markdown)
            desc_first_line = reason.rule_description.split('\n')[0]
            print(f"      {desc_first_line}")
    print()

    # Show suggested actions
    if rsv_condition_vaccinated.actions:
        print(f"Suggested Actions ({len(rsv_condition_vaccinated.actions)}):")
        for action in rsv_condition_vaccinated.actions:
            print(f"  • {action.action_type}: {action.action_description}")
            if action.url_link:
                print(f"    Link: {action.url_link}")
    else:
        print("No actions suggested (person suppressed from campaign)")

    print()
    print("=" * 60)
    print()

    # Summary
    if rsv_condition_vaccinated.status.name == "not_eligible":
        print("❌ RESULT: Person is NOT ELIGIBLE")
        print("   Reason: Already vaccinated - suppressed from campaign")
        if rsv_condition_vaccinated.actions:
            print(f"   Default action provided: {rsv_condition_vaccinated.actions[0].action_type}")
    elif rsv_condition_vaccinated.status.name == "not_actionable":
        print("⚠️  RESULT: Person is NOT ACTIONABLE")
    else:
        print("✅ RESULT: Person is ELIGIBLE")
else:
    print("❌ No conditions returned")

Eligibility Calculation Results - Vaccinated Person

Condition: RSV
Status: NOT_ELIGIBLE
Status Text: We do not believe you can have it

Rules Evaluated:

Suggested Actions (1):
  • INFO: Learn more about RSV vaccination
    Link: https://www.nhs.uk/conditions/vaccinations/rsv-vaccine


❌ RESULT: Person is NOT ELIGIBLE
   Reason: Already vaccinated - suppressed from campaign
   Default action provided: INFO


## What Happened?

The **Suppression Rule** (Priority 30) matched because:
- Person has `LAST_SUCCESSFUL_DATE` within the last 25 years
- Suppression rules **stop further processing** (`RuleStop="Y"`)
- Person is marked as `NOT_ELIGIBLE`
- A default action route is provided via `DefaultNotEligibleRouting`

This prevents already-vaccinated people from seeing booking actions.

## JSON Response - Not Eligible Person

This is what the API would return for someone who is already vaccinated:

In [153]:
print("API Response (JSON) - Vaccinated Person:")
print("=" * 60)
print()

# Convert to dict for the vaccinated person
if eligibility_status_vaccinated.conditions:
    response_data_vaccinated = {
        "conditions": [
            {
                "condition": rsv_condition_vaccinated.condition_name,
                "status": rsv_condition_vaccinated.status.name,
                "statusText": rsv_condition_vaccinated.status_text,
                "reasons": [
                    {
                        "ruleType": reason.rule_type.value,
                        "ruleName": reason.rule_name,
                        "matched": reason.matcher_matched,
                        "description": reason.rule_description
                    }
                    for reason in rsv_condition_vaccinated.suitability_rules
                ],
                "suggestedActions": [
                    {
                        "actionType": action.action_type,
                        "actionCode": action.action_code,
                        "description": action.action_description,
                        "link": str(action.url_link) if action.url_link else None,
                        "linkLabel": action.url_label if action.url_label else None
                    }
                    for action in (rsv_condition_vaccinated.actions or [])
                ]
            }
        ]
    }

    print(json.dumps(response_data_vaccinated, indent=2))
    print()
    print("✓ HTTP 200 with Content-Type: application/json")
    print("✓ Status: not_eligible")
    print("✓ Person suppressed from campaign (already vaccinated)")
else:
    print("No conditions to display")

API Response (JSON) - Vaccinated Person:

{
  "conditions": [
    {
      "condition": "RSV",
      "status": "not_eligible",
      "statusText": "We do not believe you can have it",
      "reasons": [],
      "suggestedActions": [
        {
          "actionType": "INFO",
          "actionCode": "INFO_RSV",
          "description": "Learn more about RSV vaccination",
          "link": "https://www.nhs.uk/conditions/vaccinations/rsv-vaccine",
          "linkLabel": "Learn more"
        }
      ]
    }
  ]
}

✓ HTTP 200 with Content-Type: application/json
✓ Status: not_eligible
✓ Person suppressed from campaign (already vaccinated)


### Important Note About Reasons Array

⚠️ **The `reasons` array is empty for `not_eligible` status - this is correct!**

The API only populates `suitabilityRules` when status is **`not_actionable`** (suppression rules with messaging).

For `not_eligible` status (filter rules excluding someone), the reasons are tracked internally for audit purposes but are not included in the public API response.

**API Behavior:**
- `not_eligible` → `suitabilityRules: []` (empty)
- `not_actionable` → `suitabilityRules: [...]` (contains suppression rule messages)
- `actionable` → `suitabilityRules: []` (empty)

---

# Example 3: Not Actionable Person (Suppressed with Message)

Now let's create a scenario where someone is **not actionable** - they pass filter rules but hit a suppression rule with a user-facing message. This will show when `suitabilityRules` gets populated!

## Modified Campaign - Without Vaccination Filter

To demonstrate `not_actionable`, we need to remove the filter rule that checks vaccination status, keeping only the suppression rule.

In [154]:
print("Creating Modified RSV Campaign - Not Actionable Scenario")
print("=" * 60)
print()

# Create rules WITHOUT the vaccination filter (Priority 20)
# This allows vaccinated people to pass filter checks but hit suppression
rules_not_actionable = [
    # Filter rule: Age >= 75
    IterationRule(
        Name="Remove under 75 Years",
        Type=RuleType.filter,
        Priority=10,
        AttributeName="DATE_OF_BIRTH",
        Operator=RuleOperator.year_gt,
        Comparator="-75",
        AttributeTarget="",
        Description="Ensure anyone who has a DOB determining age < 75 is filtered out",
        AttributeLevel=RuleAttributeLevel.PERSON,
        CohortLabel="RSV_COHORT",
        RuleStop="N"
    ),
    # *** REMOVED FILTER RULE FOR VACCINATION CHECK ***
    # This means vaccinated people will pass filter stage
    
    # Suppression rule: Already vaccinated (with user-facing message)
    IterationRule(
        Name="AlreadyVaccinated",
        Type=RuleType.suppression,
        Priority=30,
        AttributeName="LAST_SUCCESSFUL_DATE",
        Operator=RuleOperator.year_gte,
        Comparator="-25",
        AttributeTarget="RSV",
        Description="## You've had your RSV vaccination\n\nOur records show you were vaccinated against RSV.",
        AttributeLevel=RuleAttributeLevel.TARGET,
        RuleStop="Y"
    ),
    # Redirect rule: Leeds area
    IterationRule(
        Name="LeedsAreaContactGP",
        Type=RuleType.redirect,
        Priority=100,
        AttributeName="POSTCODE",
        Operator=RuleOperator.starts_with,
        Comparator="LS",
        AttributeTarget="",
        Description="Leeds residents should contact their GP",
        AttributeLevel=RuleAttributeLevel.PERSON,
        CommsRouting="CONTACT_GP_LEEDS",
        RuleStop="N"
    )
]

# Create iteration with modified rules
iteration_not_actionable = Iteration(
    ID="RSV_2024_ITER_002",
    Version=1,
    Name="RSV_Winter_2024_NotActionable",
    IterationDate=iteration_date,
    Type="A",
    DefaultCommsRouting="INFO_RSV",
    DefaultNotEligibleRouting="INFO_RSV",
    DefaultNotActionableRouting="INFO_RSV",
    IterationCohorts=[
        IterationCohort(
            CohortLabel="RSV_COHORT",
            CohortGroup="Older Adults"
        )
    ],
    IterationRules=rules_not_actionable,
    ActionsMapper=actions_mapper
)

# Create campaign config
campaign_config_not_actionable = CampaignConfig(
    ID="RSV_WINTER_2024_NA",
    Version=1,
    Name="RSV_WINTER_2024_NotActionable",
    Type="V",
    Target="RSV",
    IterationFrequency="A",
    IterationType="A",
    StartDate=campaign_start,
    EndDate=campaign_end,
    Iterations=[iteration_not_actionable]
)

print("Modified Campaign Configuration Created:")
print(f"  Difference: Removed filter rule for vaccination check")
print(f"  Rules: {len(rules_not_actionable)} (vs {len(rules)} in original)")
print()
print("Rules:")
for rule in rules_not_actionable:
    print(f"  • {rule.name} ({rule.type.value}) - Priority {rule.priority}")
print()
print("✓ Modified campaign configuration created")

Creating Modified RSV Campaign - Not Actionable Scenario

Modified Campaign Configuration Created:
  Difference: Removed filter rule for vaccination check
  Rules: 3 (vs 4 in original)

Rules:
  • Remove under 75 Years (F) - Priority 10
  • AlreadyVaccinated (S) - Priority 30
  • LeedsAreaContactGP (R) - Priority 100

✓ Modified campaign configuration created


## Calculate Eligibility with Modified Campaign

In [155]:
print("Running Eligibility Calculation - Not Actionable Scenario")
print("=" * 60)
print()

# Use the same vaccinated person from Example 2
# but with the modified campaign that has no vaccination filter
calculator_not_actionable = EligibilityCalculatorFactory.get(
    person=person_vaccinated,
    campaign_configs=[campaign_config_not_actionable]
)

# Get eligibility status
eligibility_status_not_actionable = calculator_not_actionable.get_eligibility_status(
    include_actions="Y",
    conditions=["RSV"],
    category="VACCINATIONS"
)

print("✓ Eligibility calculated with modified campaign")
print()

Running Eligibility Calculation - Not Actionable Scenario

✓ Eligibility calculated with modified campaign



## Examine Results - Not Actionable

In [157]:
print("Eligibility Calculation Results - Not Actionable")
print("=" * 60)
print()

if eligibility_status_not_actionable.conditions:
    rsv_condition_not_actionable = eligibility_status_not_actionable.conditions[0]

    print(f"Condition: {rsv_condition_not_actionable.condition_name}")
    print(f"Status: {rsv_condition_not_actionable.status.name.upper()}")
    print()

    # Show which rules were evaluated
    print("Rules Evaluated (suitability_rules):")
    if rsv_condition_not_actionable.suitability_rules:
        for reason in rsv_condition_not_actionable.suitability_rules:
            matched_symbol = "✓" if reason.matcher_matched else "✗"
            matched_text = "MATCHED" if reason.matcher_matched else "NOT MATCHED"
            print(f"  {matched_symbol} {reason.rule_name} ({reason.rule_type.value}): {matched_text}")
            if reason.rule_description:
                # Show first line of description
                desc_first_line = reason.rule_description.split('\n')[0]
                print(f"      {desc_first_line}")
    else:
        print("  (empty)")
    print()

    # Show suggested actions
    if rsv_condition_not_actionable.actions:
        print(f"Suggested Actions ({len(rsv_condition_not_actionable.actions)}):")
        for action in rsv_condition_not_actionable.actions:
            print(f"  • {action.action_type}: {action.action_description}")
            if action.url_link:
                print(f"    Link: {action.url_link}")
    else:
        print("No actions suggested")

    print()
    print("=" * 60)
    print()

    # Summary
    if rsv_condition_not_actionable.status.name == "not_actionable":
        print("⚠️  RESULT: Person is NOT ACTIONABLE")
        print("   Reason: Passed filter rules but suppressed with message")
        print(f"   Suitability rules populated: {len(rsv_condition_not_actionable.suitability_rules)} rule(s)")
    elif rsv_condition_not_actionable.status.name == "not_eligible":
        print("❌ RESULT: Person is NOT ELIGIBLE")
    else:
        print("✅ RESULT: Person is ELIGIBLE")
else:
    print("❌ No conditions returned")

Eligibility Calculation Results - Not Actionable

Condition: RSV
Status: NOT_ACTIONABLE

Rules Evaluated (suitability_rules):
  ✓ AlreadyVaccinated (S): MATCHED
      ## You've had your RSV vaccination

Suggested Actions (1):
  • INFO: Learn more about RSV vaccination
    Link: https://www.nhs.uk/conditions/vaccinations/rsv-vaccine


⚠️  RESULT: Person is NOT ACTIONABLE
   Reason: Passed filter rules but suppressed with message
   Suitability rules populated: 1 rule(s)


## JSON Response - Not Actionable Person

In [158]:
print("API Response (JSON) - Not Actionable Person:")
print("=" * 60)
print()

# Convert to dict for the not actionable person
if eligibility_status_not_actionable.conditions:
    response_data_not_actionable = {
        "conditions": [
            {
                "condition": rsv_condition_not_actionable.condition_name,
                "status": rsv_condition_not_actionable.status.name,
                "reasons": [
                    {
                        "ruleType": reason.rule_type.value,
                        "ruleName": reason.rule_name,
                        "matched": reason.matcher_matched,
                        "description": reason.rule_description
                    }
                    for reason in rsv_condition_not_actionable.suitability_rules
                ],
                "suggestedActions": [
                    {
                        "actionType": action.action_type,
                        "actionCode": action.action_code,
                        "description": action.action_description,
                        "link": str(action.url_link) if action.url_link else None,
                        "linkLabel": action.url_label if action.url_label else None
                    }
                    for action in (rsv_condition_not_actionable.actions or [])
                ]
            }
        ]
    }

    print(json.dumps(response_data_not_actionable, indent=2))
    print()
    print("✓ HTTP 200 with Content-Type: application/json")
    print("✓ Status: not_actionable")
    print(f"✓ Suitability Rules: {len(rsv_condition_not_actionable.suitability_rules)} rule(s) with messages")
    print("✓ This is when reasons/suitabilityRules are populated!")
else:
    print("No conditions to display")

API Response (JSON) - Not Actionable Person:

{
  "conditions": [
    {
      "condition": "RSV",
      "status": "not_actionable",
      "reasons": [
        {
          "ruleType": "S",
          "ruleName": "AlreadyVaccinated",
          "matched": true,
          "description": "## You've had your RSV vaccination\n\nOur records show you were vaccinated against RSV."
        }
      ],
      "suggestedActions": [
        {
          "actionType": "INFO",
          "actionCode": "INFO_RSV",
          "description": "Learn more about RSV vaccination",
          "link": "https://www.nhs.uk/conditions/vaccinations/rsv-vaccine",
          "linkLabel": "Learn more"
        }
      ]
    }
  ]
}

✓ HTTP 200 with Content-Type: application/json
✓ Status: not_actionable
✓ Suitability Rules: 1 rule(s) with messages
✓ This is when reasons/suitabilityRules are populated!


## Audit Record - Not Actionable

The system creates detailed audit records for each eligibility check. Let's examine what was logged:

In [159]:
print("Audit Record for Not Actionable Eligibility Check")
print("=" * 60)
print()

# Access the audit log from Flask's g object
audit_record = g.audit_log

print("Audit Event Details:")
print(f"  Request Timestamp: {audit_record.request.request_timestamp}")
print(f"  NHS Number: {audit_record.request.nhs_number}")
print(f"  Category: {audit_record.request.query_params.category}")
print(f"  Conditions Requested: {audit_record.request.query_params.conditions}")
print(f"  Include Actions: {audit_record.request.query_params.include_actions}")
print()

# Check request headers
if audit_record.request.headers.x_request_id:
    print(f"  X-Request-ID: {audit_record.request.headers.x_request_id}")
if audit_record.request.headers.x_correlation_id:
    print(f"  X-Correlation-ID: {audit_record.request.headers.x_correlation_id}")

print()

if audit_record.response.condition:
    print(f"Conditions Evaluated: {len(audit_record.response.condition)}")
    print()
    
    for condition in audit_record.response.condition:
        print(f"Condition: {condition.condition_name}")
        print(f"  Campaign ID: {condition.campaign_id}")
        print(f"  Campaign Version: {condition.campaign_version}")
        print(f"  Iteration ID: {condition.iteration_id}")
        print(f"  Status: {condition.status}")
        print()
        
        # Show filter rules evaluated
        if condition.filter_rules:
            print(f"  Filter Rules Evaluated ({len(condition.filter_rules)}):")
            for rule in condition.filter_rules:
                print(f"    • Priority {rule.rule_priority}: {rule.rule_name}")
        else:
            print("  Filter Rules: None")
        print()
        
        # Show suitability/suppression rules
        if condition.suitability_rules:
            print(f"  Suitability/Suppression Rules ({len(condition.suitability_rules)}):")
            for rule in condition.suitability_rules:
                print(f"    • Priority {rule.rule_priority}: {rule.rule_name}")
                if rule.rule_message:
                    # Show first line of message
                    first_line = rule.rule_message.split('\n')[0]
                    print(f"      Message: {first_line}")
        else:
            print("  Suitability Rules: None")
        print()
        
        # Show redirect/action rules
        if condition.action_rule:
            print(f"  Action/Redirect Rule:")
            print(f"    • Priority {condition.action_rule.rule_priority}: {condition.action_rule.rule_name}")
        else:
            print("  Action Rule: None")
        print()
        
        # Show actions
        if condition.actions:
            print(f"  Actions Returned ({len(condition.actions)}):")
            for action in condition.actions:
                print(f"    • {action.action_code}: {action.action_type}")
                if action.action_description:
                    # Show first 60 chars
                    desc = action.action_description[:60] + "..." if len(action.action_description) > 60 else action.action_description
                    print(f"      {desc}")
        else:
            print("  Actions: None")
        print()
        
        # Show cohort results
        if condition.eligibility_cohort_groups:
            print(f"  Cohort Results ({len(condition.eligibility_cohort_groups)}):")
            for cohort in condition.eligibility_cohort_groups:
                print(f"    • {cohort.cohort_code}: {cohort.cohort_status}")
                if cohort.cohort_text:
                    print(f"      Text: {cohort.cohort_text}")
        
        print()
        print("=" * 60)
else:
    print("No conditions in audit record")

print()
print("✓ Audit record captures all rule evaluations for compliance/debugging")

Audit Record for Not Actionable Eligibility Check

Audit Event Details:
  Request Timestamp: 2025-10-12 15:41:32.554171+00:00
  NHS Number: None
  Category: None
  Conditions Requested: None
  Include Actions: None


Conditions Evaluated: 3

Condition: RSV
  Campaign ID: RSV_WINTER_2024
  Campaign Version: 1
  Iteration ID: RSV_2024_ITER_001
  Status: actionable

  Filter Rules: None

  Suitability Rules: None

  Action/Redirect Rule:
    • Priority 100: LeedsAreaContactGP

  Actions Returned (1):
    • CONTACT_GP_RSV: CONTACT
      Contact your GP surgery to arrange RSV vaccination

  Cohort Results (1):
    • Older Adults: actionable

Condition: RSV
  Campaign ID: RSV_WINTER_2024
  Campaign Version: 1
  Iteration ID: RSV_2024_ITER_001
  Status: not_eligible

  Filter Rules Evaluated (1):
    • Priority 20: Remove from RSV cohort if already vaccinated

  Suitability Rules: None

  Action Rule: None

  Actions Returned (1):
    • INFO_RSV: INFO
      Learn more about RSV vaccination

 

## Audit Record Structure (JSON)

Let's also view the complete audit record structure as JSON:

In [160]:
print("Complete Audit Record Structure (JSON)")
print("=" * 60)
print()

# Convert audit record to dict for JSON serialization
# Using model_dump to get the full structure
audit_dict = audit_record.model_dump(mode='json', exclude_none=False)

# Pretty print the JSON structure
print(json.dumps(audit_dict, indent=2, default=str))

print()
print("=" * 60)
print()
print("✓ This shows the complete internal audit structure")
print("✓ Note: request and response are nested objects")
print("✓ All rule evaluations are captured for compliance")

Complete Audit Record Structure (JSON)

{
  "request": {
    "request_timestamp": "2025-10-12T15:41:32.554171Z",
    "headers": {
      "x_request_id": null,
      "x_correlation_id": null,
      "nhsd_end_user_organisation_ods": null,
      "nhsd_application_id": null
    },
    "query_params": {
      "category": null,
      "conditions": null,
      "include_actions": null
    },
    "nhs_number": null
  },
  "response": {
    "response_id": null,
    "last_updated": null,
    "condition": [
      {
        "campaign_id": "RSV_WINTER_2024",
        "campaign_version": 1,
        "iteration_id": "RSV_2024_ITER_001",
        "iteration_version": 1,
        "condition_name": "RSV",
        "status": "actionable",
        "status_text": "You should have the RSV vaccine",
        "eligibility_cohorts": [
          {
            "cohort_code": "RSV_COHORT",
            "cohort_status": "actionable"
          }
        ],
        "eligibility_cohort_groups": [
          {
            "coho

### Audit Insights

**What the audit record shows:**

1. **Filter Rules** - Which filter rules were evaluated (Age check)
2. **Suitability Rules** - Which suppression rules matched (AlreadyVaccinated with full message)
3. **Redirect Rules** - Which redirect rules were evaluated (Leeds area check)
4. **Actions** - What actions were ultimately provided to the user
5. **Cohort Results** - How each cohort was evaluated

**Key difference from Example 2:**
- Example 2 (not_eligible): Filter rule excluded person → no suitability rules in audit
- Example 3 (not_actionable): Passed filters, suppression matched → **suitability rules captured in audit with full message**

This audit trail is crucial for:
- **Compliance**: Understanding why someone received specific guidance
- **Debugging**: Tracking rule evaluation logic
- **Analytics**: Understanding campaign performance

## Key Difference: Filter vs Suppression

**Example 2 (Not Eligible):**
- Had **filter rule** checking vaccination (Priority 20)
- Person excluded at filter stage → `not_eligible`
- `suitabilityRules: []` (empty)

**Example 3 (Not Actionable):**
- **Removed filter rule** for vaccination
- Person passes filter stage but hits **suppression rule** (Priority 30)
- Status: `not_actionable`
- `suitabilityRules: [...]` **populated with suppression rule message!**

This shows how the same person (vaccinated) gets different API responses depending on whether they're excluded by a filter or suppression rule.

---

# Summary

## What We Demonstrated

1. **Request Validation** ✓
   - Valid request passes all checks
   - NHS number mismatch → 401 Unauthorized
   - Invalid query params → 400 Bad Request

2. **Data Setup** ✓
   - Created Person records:
     - **Eligible**: 76yo, Leeds, RSV cohort, not vaccinated
     - **Not Eligible/Not Actionable**: 78yo, Manchester, RSV cohort, vaccinated 6 months ago
   - Created two RSV Campaign configurations (with/without vaccination filter)

3. **Eligibility Calculation - Example 1 (Actionable)** ✓
   - All filter rules passed
   - Person is **actionable**
   - Leeds-specific action applied (Contact GP)
   - `suitabilityRules: []` (empty for actionable)

4. **Eligibility Calculation - Example 2 (Not Eligible)** ✓
   - **Filter rule** excluded person (vaccinated within 25 years)
   - Status: **not_eligible**
   - `suitabilityRules: []` (empty - filter exclusions not exposed)
   - Default not-eligible action provided

5. **Eligibility Calculation - Example 3 (Not Actionable)** ✓
   - Passed filter rules (no vaccination filter in modified campaign)
   - **Suppression rule** matched with user message
   - Status: **not_actionable**
   - `suitabilityRules: [...]` **POPULATED with suppression rule message!**
   - Shows when/how suitability rules appear in API response

## Key Takeaways

- **Request validation** happens before any business logic
- **Filter rules** determine who is in scope (exclusions not shown in API)
- **Suppression rules** provide user-facing messages (shown in `suitabilityRules`)
- **Redirect rules** customize actions based on attributes (location, etc.)
- **suitabilityRules only populated for `not_actionable` status**
- Rules are evaluated **sequentially by priority**
- The system is **fully type-safe** and uses production data structures

## Status vs SuitabilityRules Mapping

| Status | suitabilityRules | Why |
|--------|------------------|-----|
| `actionable` | `[]` (empty) | Person eligible, no explanation needed |
| `not_eligible` | `[]` (empty) | Filter exclusion, not exposed to users |
| `not_actionable` | `[...]` (populated) | Suppression with user-facing message |

## Rule Types Demonstrated

| Type | Priority | Purpose | Exposed in API |
|------|----------|---------|----------------|
| Filter (F) | 10-20 | Include/exclude from campaign | No - internal only |
| Suppression (S) | 30+ | Stop with user message | Yes - in suitabilityRules |
| Redirect (R) | 100+ | Route to specific actions | Yes - as actions |

---

**End of Demo**